In [ ]:
# import packages
import sys
import numpy as np
from pathlib import Path
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
from scipy.signal import correlate
import pandas as pd
from math import factorial
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)
from src.d00_utils import utilities as utils
from src.d00_utils.dirnames import masked_imgs_dirname, corr_dirname, proc_dirname, analyses_dirname

from scipy.stats import ttest_rel

In [ ]:
input_dirpath = '/Users/kwu2/Library/CloudStorage/GoogleDrive-kwu2@stanford.edu/My Drive/Lab/CryoEM_live_imaging/Experiments/CE012/CZI_to_process/div7_posttimelapse_and_div7_fixed_stained_aligned/img_processing/masked_imgs/raw_ometifs_masked_cellmasks_no_nuc'

In [ ]:
caax_live_ch = 0
caax_fix_ch = 2
cell_ch = 1
MBP_ch = 3

ch_labels = {caax_live_ch: 'CAAX (live)',
             caax_fix_ch: 'CAAX (fixed)', 
             cell_ch: 'cell membrane', MBP_ch: 'MBP'}

In [ ]:
input_dirpath = Path(input_dirpath)

proc_dirpath = utils.get_proc_dirpath(input_dirpath)
corr_dirpath = proc_dirpath / analyses_dirname / corr_dirname
corr_dirpath.mkdir(parents=True, exist_ok=True)

input_dirname = input_dirpath.name
analysis_name = f'p_coeff_{input_dirname}'
p_coeff_path = corr_dirpath / f'{analysis_name}.csv'

In [ ]:
# Calculate correlation coefficient
imgpaths = [path for path in Path(input_dirpath).glob('*.ome.tif')]

def norm_data(data):
    """
    normalize data to have mean=0 and standard_deviation=1
    """
    mean_data=np.mean(data)
    std_data=np.std(data, ddof=1)
    #return (data-mean_data)/(std_data*np.sqrt(data.size-1))
    return (data-mean_data)/(std_data)

def show_images(imgname, ch1_img, ch2_img, combined_img, p_coeff):
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(ch1_img, cmap='gray')
    axs[0].set_title(f'Ch {ch1} img')
    axs[0].axis('off')
    
    axs[1].imshow(ch2_img, cmap='gray')
    axs[1].set_title(f'Ch {ch2} img')
    axs[1].axis('off')
    
    axs[2].imshow(combined_img, cmap='gray')
    axs[2].set_title('Mask')
    axs[2].axis('off')
    fig.suptitle(f'{imgname}\n p-coeff: {p_coeff}')
    plt.tight_layout()
    plt.show()
    

# Calculate Pearson coefficient
def calculate_p_coeff(ch1_img, ch2_img):
    ch1_norm = (norm_data(ch1_img)).flatten()
    ch2_norm = (norm_data(ch2_img)).flatten()
    
    cov_matrix = np.corrcoef(ch1_norm, ch2_norm)
    p_coeff = cov_matrix[0, 1]
    return p_coeff

p_coeff_table = pd.DataFrame()
for imgpath in imgpaths:
    
    img_file = AICSImage(imgpath, reader=OmeTiffReader)
    img = img_file.data
    img_d = utils.extract_img_info(imgpath.name)
    
    # Loop through every combination of channels
    size_c = img.shape[1]
    for ch1 in range(size_c):
        for ch2 in range((ch1+1), size_c):
            ch1_img = img[:, ch1, :, :, :].squeeze()
            ch2_img = img[:, ch2, :, :, :].squeeze()
            
            combined_img = ch1_img + ch2_img
            ch1_img = ch1_img[combined_img > 0].flatten()
            ch2_img = ch2_img[combined_img > 0].flatten()
            
            p_coeff = calculate_p_coeff(ch1_img, ch2_img)
            img_d[f'{ch_labels[ch1]} and {ch_labels[ch2]}'] = [p_coeff]
    p_coeff_table = pd.concat([p_coeff_table, img_d], ignore_index=True)
                 
    #show_images(imgpath.name, ch1_img, ch2_img, combined_img, p_coeff)

p_coeff_table.to_csv(p_coeff_path)
    

In [ ]:
col_names = list(p_coeff_table.columns)
print(col_names)

In [ ]:
img_info_cols = col_names[:6]
p_coeff_dfm = p_coeff_table.melt(img_info_cols, var_name='comparison', value_name='correlation coefficients')

ax = sns.swarmplot(p_coeff_dfm, x='comparison', y='correlation coefficients', hue='UID')
ax.tick_params(axis='x', labelrotation = 90)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

plt.savefig(corr_dirpath / f'{analysis_name}.png', bbox_inches="tight")